In [1]:
import os
import copy
import wget
import time
import asyncio
import warnings
import logging

import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS

%matplotlib inline

In [2]:
attcs = ATTCS()
# cam = salobj.Remote(attcs.domain, "GenericCamera", 1)

await attcs.start_task

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 17 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 23 messages


[None, None, None, None, None, None, None]

In [3]:
# Declare target name
target_name='HD28464'

In [31]:
await attcs.slew_icrs(ra = "04 28 21.7024656911", dec = "-23 26 38.225915193", rot_pa=-180., target_name=target_name)

In [5]:
await attcs.enable(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "auxtel_201912_crazy.yaml",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

# Standard offset between pointing model and center of CCD

In [7]:
rotation_matrix = lambda angle: np.array([
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.],
            [0., 0., 1.]])
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20, -18., 0.], rotation_matrix(angle))

In [8]:
el, az

(26.883590122498866, 1.1280878181602318)

In [9]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [19]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20, -18+115., 0.], rotation_matrix(angle))

In [20]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [21]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20, -18+115+128., 0.], rotation_matrix(angle))

In [22]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+123, -18+115+128., 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [30]:
(np.random.random()-0.5)*240.

98.88878563541553

RemoteEvent(ATMCS, 0, target) falling behind; read 97 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 47 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 13 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 19 messages


In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
5*360/24

In [ ]:
attcs.slew_icrs?

In [ ]:
await attcs.slew_icrs

In [ ]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="x", offset=x_offset)
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="y", offset=y_offset)

In [ ]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)

In [ ]:
dz = 0.8

try:
    await cam.cmd_stopLiveView.start()
except salobj.AckError:
    print("ack error")

# Intrafocal hexapod setting - offsets -0.5mm from central focus value above

print('Move to intrafocal position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                              z=hex_Z-dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=-dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print('Take intrafocal image')
cam.evt_endReadout.flush()

await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_intra')

intra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

# Extrafocal hexapod setting
print('Move to extra-focal hexapod position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                                      z=hex_Z+dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

# r.evt_endReadout.flush()
print('Take extra-focal image')
cam.evt_endReadout.flush()
await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_extra')

extra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()

print(f"============")
print(f"dz={dz}")
print(intra_endron.imageName)
print(extra_endron.imageName)
print(f"============")

try:
    await cam.cmd_startLiveView.set_start(expTime=0.5)
except salobj.AckError:
    print("ack error")

attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print(f"['{intra_endron.imageName}.fits'], ['{extra_endron.imageName}.fits']")
print(np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle))

# intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
# extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
# intra_fname = wget.download(intra_wget_url)
# print(f"============")
# print(f'{intra_fname}')
# extra_fname = wget.download(extra_wget_url)
# print(f'{extra_fname}')
# print(f"============")


In [ ]:
intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'

intra_fname = wget.download(intra_wget_url, out="/home/saluser/develop/cwfs/python/data")
extra_fname = wget.download(extra_wget_url, out="/home/saluser/develop/cwfs/python/data")

print(intra_fname,extra_fname)

In [ ]:
await attcs.ataos.cmd_offset.set_start(z=0.)

In [ ]:
await attcs.ataos.cmd_resetOffset.start()

In [17]:
await attcs.ataos.cmd_offset.set_start(z=-0.1)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
print("done")

falling behind; queue contains 45 elements


done


In [18]:
await attcs.atdome.cmd_openShutter.start()